In [1]:
# check xgboost version
import xgboost
print(xgboost.__version__)
# evaluate an xgboost regression model on the housing dataset
from numpy import absolute
import pandas as pd
from sklearn.model_selection import cross_val_score, RepeatedKFold, train_test_split
from xgboost import XGBRegressor
import numpy as np

1.4.2


In [2]:
data = pd.read_csv('xtrain.csv',index_col=0)
target = pd.read_csv('train.csv')

In [3]:
# data.dropna(axis= 0, inplace=True)
data.fillna(0, inplace=True)
target = data['SalePrice']
data.drop('SalePrice',inplace=True, axis=1)
data.drop('LotFrontage', inplace=True, axis=1)
data.replace(np.inf, 0, inplace=True)

In [4]:
target = np.log(target)

In [5]:
pd.set_option('display.max_columns', None)
data.head(25)

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_Grvl,x2_No,x2_Pave,x3_IR1,x3_IR2,x3_Reg,x3_rare,x4_AllPub,x4_rare,x5_Corner,x5_CulDSac,x5_FR2,x5_Inside,x5_rare,x6_Blmngtn,x6_BrDale,x6_BrkSide,x6_ClearCr,x6_CollgCr,x6_Crawfor,x6_Edwards,x6_Gilbert,x6_IDOTRR,x6_MeadowV,x6_Mitchel,x6_NAmes,x6_NWAmes,x6_NoRidge,x6_NridgHt,x6_OldTown,x6_SWISU,x6_Sawyer,x6_SawyerW,x6_Somerst,x6_StoneBr,x6_Timber,x6_rare,x7_Gable,x7_Hip,x7_rare,x8_CompShg,x8_rare,x9_AsbShng,x9_BrkFace,x9_CemntBd,x9_HdBoard,x9_MetalSd,x9_Plywood,x9_Stucco,x9_VinylSd,x9_Wd Sdng,x9_WdShing,x9_rare,x10_AsbShng,x10_BrkFace,x10_CmentBd,x10_HdBoard,x10_MetalSd,x10_Plywood,x10_Stucco,x10_VinylSd,x10_Wd Sdng,x10_Wd Shng,x10_rare,x11_BrkCmn,x11_BrkFace,x11_None,x11_Stone,x11_rare,x12_BrkTil,x12_CBlock,x12_PConc,x12_Slab,x12_rare,x13_GasA,x13_GasW,x13_rare,x14_N,x14_Y,x15_FuseA,x15_FuseF,x15_SBrkr,x15_rare,x16_Attchd,x16_Basment,x16_BuiltIn,x16_Detchd,x16_No,x16_rare,x17_COD,x17_New,x17_WD,x17_rare,x18_Abnorml,x18_Family,x18_Normal,x18_Partial,x18_rare,x19_No,x19_Shed,x19_rare,x20_Bnk,x20_HLS,x20_Low,x20_Lvl,x21_Gtl,x21_Mod,x21_rare,x22_Artery,x22_Feedr,x22_Norm,x22_PosN,x22_RRAn,x22_rare,x23_Norm,x23_rare,x24_1Fam,x24_2fmCon,x24_Duplex,x24_Twnhs,x24_TwnhsE,x25_1.5Fin,x25_1Story,x25_2Story,x25_SFoyer,x25_SLvl,x25_rare,x26_0,x26_3,x26_4,x26_5,x26_6,x26_7,x26_8,x26_9,x26_10,x27_0,x27_3,x27_4,x27_5,x27_6,x27_7,x27_8,x27_9,x28_Ex,x28_Gd,x28_TA,x28_rare,x29_Fa,x29_Gd,x29_TA,x29_rare,x30_Ex,x30_Fa,x30_Gd,x30_No,x30_TA,x31_Fa,x31_Gd,x31_No,x31_TA,x31_rare,x32_Av,x32_Gd,x32_Mn,x32_No,x33_ALQ,x33_BLQ,x33_GLQ,x33_LwQ,x33_No,x33_Rec,x33_Unf,x34_Ex,x34_Fa,x34_Gd,x34_TA,x34_rare,x35_Ex,x35_Fa,x35_Gd,x35_No,x35_Po,x35_TA,x36_Fin,x36_No,x36_RFn,x36_Unf,x37_Fa,x37_No,x37_TA,x37_rare,x38_Fa,x38_No,x38_TA,x38_rare,x39_No,x39_rare,x40_GdPrv,x40_GdWo,x40_MnPrv,x40_No,x40_rare,x41_ALQ,x41_BLQ,x41_LwQ,x41_No,x41_Rec,x41_Unf,x41_rare,x42_Ex,x42_Fa,x42_Gd,x42_TA,x43_N,x43_P,x43_Y,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.122500,6.797940,0.413559,0.0,6.797940,0.666667,0.5,0.375,0.333333,0.000000,0.002488,0.386460,0.50,0.000000,0.111517,0.000000,0.000000,0.000000,0.0,0.090909,0.50,9.360655,0.125089,0.000000,0.064212,0.333333,0.140098,0.0,0.500000
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1

In [17]:
data.tail()

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_Grvl,x2_No,x2_Pave,...,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
1455,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.25,3.045,7.546,0.000,0.000,0.408,0.000,0.156,0.0,0.417
1456,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.00,5.075,9.903,0.140,0.111,0.252,0.333,0.252,0.0,0.417
1457,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.00,4.127,9.253,0.049,0.000,0.375,0.000,0.189,0.0,0.583
1458,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.00,4.304,9.172,0.009,0.698,0.000,0.333,0.176,0.0,0.250
1459,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.50,0.000,0.000,0.147,0.197,0.058,0.333,0.206,0.0,0.333


In [16]:
target.shape, data.shape

((1460,), (1460, 254))

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(data, target, test_size=0.3)

In [7]:
from sklearn.linear_model import LinearRegression

In [8]:
reg = LinearRegression()

In [9]:
reg.fit(xtrain,ytrain)

LinearRegression()

In [10]:
reg.score(xtest, ytest)

-7.551930679004797e+21

In [ ]:
ytrain.isnull().values.any()

In [ ]:
nan_rows = xtrain[xtrain.isnull().any(1)]
nan_rows

In [ ]:
data.dropna(axis= 1, inplace=True)


In [ ]:
data.fillna(0)

In [14]:
data = data.round(3)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Columns: 254 entries, x0_FV to TotRmsAbvGrd
dtypes: float64(254)
memory usage: 2.9 MB


In [24]:
xtrain

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_Grvl,x2_No,x2_Pave,...,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
68,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.00,4.394,8.940,0.000,0.000,0.320,0.000,0.122,0.0,0.167
829,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.00,4.673,9.521,0.000,0.000,0.257,0.000,0.098,0.0,0.167
441,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.50,4.382,9.594,0.024,0.000,0.560,0.000,0.236,0.0,0.500
785,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.75,-inf,9.797,0.044,0.427,0.210,0.000,0.225,0.0,0.417
159,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.00,4.248,8.949,0.010,0.000,0.571,0.333,0.228,0.0,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.00,3.466,8.412,0.173,0.000,0.000,0.333,0.160,0.0,0.167
1074,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.25,-inf,9.452,0.000,0.000,0.531,0.000,0.203,0.0,0.250
252,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.50,4.443,9.271,0.000,0.000,0.342,0.000,0.131,0.0,0.333
233,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.00,3.738,9.040,0.032,0.483,0.000,0.333,0.146,0.0,0.250


In [16]:
r = data.index[np.isinf(data).any(1)]
print(r)

Int64Index([   6,   40,   41,   45,   47,   52,   59,   75,   82,   98,
            ...
            1354, 1355, 1358, 1379, 1385, 1387, 1390, 1440, 1441, 1448],
           dtype='int64', length=227)


In [26]:
import numpy as np

In [11]:
reg.coef_

array([-1.17913133e+09, -1.17913133e+09, -1.17913133e+09, -1.17913133e+09,
       -1.17913133e+09, -2.45415883e+11, -2.45415883e+11,  5.63575660e+12,
        5.63575660e+12,  5.63575660e+12,  2.22671402e+12,  2.22671402e+12,
        2.22671402e+12,  2.22671402e+12, -1.37446915e+12, -1.37446915e+12,
       -2.60764389e+11, -2.60764389e+11, -2.60764389e+11, -2.60764389e+11,
       -2.60764389e+11, -4.52089692e+10, -4.52089692e+10, -4.52089692e+10,
       -4.52089692e+10, -4.52089692e+10, -4.52089692e+10, -4.52089692e+10,
       -4.52089692e+10, -4.52089692e+10, -4.52089692e+10, -4.52089692e+10,
       -4.52089692e+10, -4.52089692e+10, -4.52089692e+10, -4.52089692e+10,
       -4.52089692e+10, -4.52089692e+10, -4.52089692e+10, -4.52089692e+10,
       -4.52089692e+10, -4.52089692e+10, -4.52089692e+10, -4.52089692e+10,
        3.80342581e+12,  3.80342581e+12,  3.80342581e+12, -8.29489006e+10,
       -8.29489006e+10, -1.50450909e+12, -1.50450909e+12, -1.50450909e+12,
       -1.50450909e+12, -

In [8]:
df.columns.to_series()[np.isinf(df).any()]

LotFrontage    LotFrontage
dtype: object

In [31]:
test.columns.to_series()[np.isinf(test).any()]

Series([], dtype: object)

In [32]:
test = pd.read_csv('xtest.csv')

In [ ]:
data[data['LotFrontage']]

In [11]:
data['LotFrontage'].unique()

117

In [17]:
data.loc[r]

,x0_FV,x0_RH,x0_RL,x0_RM,x0_rare,x1_Pave,x1_rare,x2_Grvl,x2_No,x2_Pave,x3_IR1,x3_IR2,x3_Reg,x3_rare,x4_AllPub,x4_rare,x5_Corner,x5_CulDSac,x5_FR2,x5_Inside,x5_rare,x6_Blmngtn,x6_BrDale,x6_BrkSide,x6_ClearCr,x6_CollgCr,x6_Crawfor,x6_Edwards,x6_Gilbert,x6_IDOTRR,x6_MeadowV,x6_Mitchel,x6_NAmes,x6_NWAmes,x6_NoRidge,x6_NridgHt,x6_OldTown,x6_SWISU,x6_Sawyer,x6_SawyerW,x6_Somerst,x6_StoneBr,x6_Timber,x6_rare,x7_Gable,x7_Hip,x7_rare,x8_CompShg,x8_rare,x9_AsbShng,x9_BrkFace,x9_CemntBd,x9_HdBoard,x9_MetalSd,x9_Plywood,x9_Stucco,x9_VinylSd,x9_Wd Sdng,x9_WdShing,x9_rare,x10_AsbShng,x10_BrkFace,x10_CmentBd,x10_HdBoard,x10_MetalSd,x10_Plywood,x10_Stucco,x10_VinylSd,x10_Wd Sdng,x10_Wd Shng,x10_rare,x11_BrkCmn,x11_BrkFace,x11_None,x11_Stone,x11_rare,x12_BrkTil,x12_CBlock,x12_PConc,x12_Slab,x12_rare,x13_GasA,x13_GasW,x13_rare,x14_N,x14_Y,x15_FuseA,x15_FuseF,x15_SBrkr,x15_rare,x16_Attchd,x16_Basment,x16_BuiltIn,x16_Detchd,x16_No,x16_rare,x17_COD,x17_New,x17_WD,x17_rare,x18_Abnorml,x18_Family,x18_Normal,x18_Partial,x18_rare,x19_No,x19_Shed,x19_rare,x20_Bnk,x20_HLS,x20_Low,x20_Lvl,x21_Gtl,x21_Mod,x21_rare,x22_Artery,x22_Feedr,x22_Norm,x22_PosN,x22_RRAn,x22_rare,x23_Norm,x23_rare,x24_1Fam,x24_2fmCon,x24_Duplex,x24_Twnhs,x24_TwnhsE,x25_1.5Fin,x25_1Story,x25_2Story,x25_SFoyer,x25_SLvl,x25_rare,x26_0,x26_3,x26_4,x26_5,x26_6,x26_7,x26_8,x26_9,x26_10,x27_0,x27_3,x27_4,x27_5,x27_6,x27_7,x27_8,x27_9,x28_Ex,x28_Gd,x28_TA,x28_rare,x29_Fa,x29_Gd,x29_TA,x29_rare,x30_Ex,x30_Fa,x30_Gd,x30_No,x30_TA,x31_Fa,x31_Gd,x31_No,x31_TA,x31_rare,x32_Av,x32_Gd,x32_Mn,x32_No,x33_ALQ,x33_BLQ,x33_GLQ,x33_LwQ,x33_No,x33_Rec,x33_Unf,x34_Ex,x34_Fa,x34_Gd,x34_TA,x34_rare,x35_Ex,x35_Fa,x35_Gd,x35_No,x35_Po,x35_TA,x36_Fin,x36_No,x36_RFn,x36_Unf,x37_Fa,x37_No,x37_TA,x37_rare,x38_Fa,x38_No,x38_TA,x38_rare,x39_No,x39_rare,x40_GdPrv,x40_GdWo,x40_MnPrv,x40_No,x40_rare,x41_ALQ,x41_BLQ,x41_LwQ,x41_No,x41_Rec,x41_Unf,x41_rare,x42_Ex,x42_Fa,x42_Gd,x42_TA,x43_N,x43_P,x43_Y,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
6,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.116250,7.079184,0.000000,0.000000,7.079184,0.666667,0.0,0.375,0.333333,0.333333,0.001493,0.448519,0.50,0.297550,0.104205,0.0,0.0,0.0,0.0,0.636364,0.25,-inf,8.984694,0.242558,0.000000,0.135702,0.333333,0.275941,0.0,0.416667
40,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0